## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-20-51-00 +0000,wsj,Texas Teachers Union Sues Education Agency Ove...,https://www.wsj.com/us-news/education/texas-te...
1,2026-01-06-20-48-18 +0000,nyt,Georgia Schedules Special Election for Marjori...,https://www.nytimes.com/2026/01/06/us/georgia-...
2,2026-01-06-20-47-27 +0000,nyt,Live Updates: Venezuela Projects Unity as Trum...,https://www.nytimes.com/live/2026/01/06/world/...
3,2026-01-06-20-47-24 +0000,nyt,How Venezuela’s Oil Could Shake Up the World S...,https://www.nytimes.com/2026/01/06/business/ve...
4,2026-01-06-20-45-43 +0000,nyt,"Democrats Commemorate Jan. 6, Highlighting Tru...",https://www.nytimes.com/2026/01/06/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,55
20,venezuela,55
161,maduro,40
29,oil,20
127,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
322,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,172
133,2026-01-06-14-57-22 +0000,bbc,Trump claims US oil firms could be 'up and run...,https://www.bbc.com/news/articles/c4grxzxjjd8o...,154
244,2026-01-06-03-03-09 +0000,nypost,Trump offers first timeline for expanded US oi...,https://nypost.com/2026/01/05/us-news/trump-of...,144
22,2026-01-06-20-13-46 +0000,nypost,Trump floats reimbursing US oil companies to r...,https://nypost.com/2026/01/06/business/trump-f...,144
57,2026-01-06-19-09-13 +0000,wapo,Venezuela’s Machado gushes over Trump while ca...,https://www.washingtonpost.com/world/2026/01/0...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
322,172,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
44,70,2026-01-06-19-39-00 +0000,wsj,Trump’s insistence that the U.S. should take c...,https://www.wsj.com/politics/policy/trump-gree...
317,67,2026-01-05-21-36-00 +0000,wsj,U.S. companies will benefit under a new global...,https://www.wsj.com/politics/policy/oecd-corpo...
174,52,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
252,49,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
279,46,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
120,39,2026-01-06-16-16-53 +0000,bbc,Swiss ski bar not inspected for five years bef...,https://www.bbc.com/news/articles/cdx4zxk1np0o...
200,37,2026-01-06-10-00-08 +0000,wapo,"Trump team puts a target on Cuba, with threats...",https://www.washingtonpost.com/national-securi...
125,36,2026-01-06-15-40-33 +0000,nypost,Hilton stock dips after DHS accuses hotel chai...,https://nypost.com/2026/01/06/business/hilton-...
262,34,2026-01-06-01-37-29 +0000,nypost,Newsom claims he was ‘physically’ in middle of...,https://nypost.com/2026/01/05/us-news/newsom-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
